In [ ]:
import os
# Version for filename
ver = 'GRU'  # GRU or LSTM

date = "mother_folder"
dl_folder = f"base_path/{date}_DL"

dl_number = "project_name"
#os.mkdir(f"{dl_folder}/model_{dl_number}")

In [ ]:
fs = 20  # sampling rate
look_frame = 30  

In [3]:
# Ignore warning
import os
import tensorflow as tf
import logging
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)

import numpy
#import pandas
import matplotlib.pyplot as plt
from scipy import io
from tensorflow.keras import layers, losses, optimizers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, ConvLSTM2D, Bidirectional
from sklearn.metrics import roc_curve
from sklearn.metrics import auc, roc_auc_score

In [4]:
# Allow grouth option of GPU
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [5]:
# Clear session
from keras import backend as K
K.clear_session()

Using TensorFlow backend.


In [ ]:
import tqdm
import numpy as np

val_loss_list = []
for n in tqdm.tqdm(range(5)):
    model_dir = os.path.join(os.path.join(dl_folder, f"model_{dl_number}"), f"model_{n}")
    val_loss = np.load(os.path.join(model_dir, f"valid_loss.npy"))
    val_loss_list.append(val_loss)

val_loss_list = np.array(val_loss_list)

print(val_loss_list.shape)

100%|██████████| 5/5 [00:00<00:00, 97.75it/s]

(5, 30)


In [ ]:
import numpy as np
import tqdm

def _predict_for_score(model, X):
    # sklearn
    if hasattr(model, "predict_proba"):
        proba = np.asarray(model.predict_proba(X))

        if proba.ndim == 2 and proba.shape[1] == 1:
            return proba[:, 0]  

        if proba.ndim == 2 and proba.shape[1] >= 2:
            return proba[:, 1]
        return proba.ravel()

    if hasattr(model, "decision_function"):
        return np.asarray(model.decision_function(X)).ravel()


    yhat = np.asarray(model.predict(X))
    return yhat.ravel()

def permutation_importance_time_feature(
    model, X, y, score_fn, n_repeats=5, random_state=0
):
    rng = np.random.default_rng(random_state)
    X = np.asarray(X)
    y = np.asarray(y)

    base_pred = _predict_for_score(model, X)
    baseline = score_fn(y, base_pred)

    n, T, F = X.shape
    drops = np.zeros((T, F, n_repeats), dtype=float)

    for t in range(T):
        for f in range(F):
            for r in range(n_repeats):
                perm = rng.permutation(n)
                Xp = X.copy()
                Xp[:, t, f] = X[perm, t, f]  
                pred = _predict_for_score(model, Xp)
                drops[t, f, r] = baseline - score_fn(y, pred)

    return baseline, drops.mean(axis=2), drops.std(axis=2)


In [ ]:
baselines = []
importance, std_imp = [], []
for n in tqdm.tqdm(range(5)):
    model_dir = os.path.join(os.path.join(dl_folder, f"model_{dl_number}"), f"model_{n}")


    epoch = val_loss_list[n].argmin() + 1
    if epoch > 9:
        model_file_path = os.path.join(
            model_dir,
            f"{date}_{dl_number}_ver{ver}_epoch{epoch}.h5"
        )

    if epoch < 10:
        model_file_path = os.path.join(
            model_dir,
            f"{date}_{dl_number}_ver{ver}_epoch0{epoch}.h5"
        )        

    model = load_model(model_file_path)

    input_valid = np.load(f"{model_dir}/{date}_{dl_number}_valid_features.npy")
    validY = np.load(f"{model_dir}/{date}_{dl_number}_valid_targets.npy")

    baseline, drops_mean, drops_std = permutation_importance_time_feature(model, input_valid, validY, score_fn=roc_auc_score, n_repeats=3, random_state=0)

    baselines.append(baseline)
    importance.append(drops_mean)
    std_imp.append(drops_std)


baselines = np.array(baselines)
importance = np.array(importance)
std_imp = np.array(std_imp)

print(baselines.shape)
print(importance.shape, std_imp.shape)

100%|██████████| 5/5 [14:35<00:00, 175.04s/it]

(5,)
(5, 31, 10) (5, 31, 10)


In [9]:
np.save((f"{dl_folder}/model_{dl_number}/{date}_{dl_number}_valid_pemutation_importance.npy"), importance)
np.save((f"{dl_folder}/model_{dl_number}/{date}_{dl_number}_valid_pemutation_importance_std.npy"), std_imp)
np.save((f"{dl_folder}/model_{dl_number}/{date}_{dl_number}_valid_pemutation_importance_baseline.npy"), baselines)